In [28]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

pyLDAvis.enable_notebook()
nltk.download('wordnet')
nltk.download('stopwords')
#sklearn.decomposition.LatentDirichletAllocation

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
data = pd.read_csv('data.csv')
data.fillna('', inplace=True)
data.head()

,contest,problem_name,problem_statement,problem_tags
0,325,A,You are given n rectangles. The corners of rec...,"implementation,*1500"
1,325,B,Daniel is organizing a football tournament. He...,"binarysearch,math,*1800"
2,325,C,Piegirl has found a monster and a book about m...,"dfsandsimilar,graphs,shortestpaths,*2600"
3,325,D,"In a far away land, there exists a planet shap...","dsu,*2900"
4,325,E,Piegirl found the red button. You have one las...,"combinatorics,dfsandsimilar,dsu,graphs,greedy,..."


In [3]:
print(f'quantidade de problemas: {data.shape[0]}')
print(f'quantidade de contests: {len(data.contest.unique())}')
print(f'problemas repetidos: {data.shape[0] - len(data.problem_statement.unique())}')

data.drop_duplicates(subset='problem_statement', inplace=True)

quantidade de problemas: 8343
quantidade de contests: 1424
problemas repetidos: 1524


## Pre-processamento

In [4]:
sw = stopwords.words('english')
sw.append(['input', 'output', 'th'])
lemma = WordNetLemmatizer()
text = data.problem_statement

# Problem_statement
text = text.apply(lambda x: re.sub('[,|.|(|)|$|!|?|!]',' ',x)) # removendo caracteres especiais
text = text.apply(lambda x: [i for i in x.split() if i.lower() not in sw]) # removendo stop-words
text = text.apply(lambda x: ' '.join([lemma.lemmatize(i) for i in x])) # lemmatizando todo

# Topics
topics = data.problem_tags
topics = topics.apply(lambda x: re.sub('[*][0-9]+','',x)) # remove os ratings *800 etc
topics = topics.apply(lambda x: [i for i in x.split(',') if i != '']) # tira strings vazias

In [5]:
text

0       given n rectangle corner rectangle integer coo...
1       Daniel organizing football tournament come fol...
2       Piegirl found monster book monster pie reading...
3       far away land exists planet shaped like cylind...
4       Piegirl found red button one last chance chang...
                              ...                        
8338    n block arranged row numbered left right start...
8339    map capital Berland viewed infinite coordinate...
8340    play strategic video game yeah ran good proble...
8341    first let's define function f x follows: \begi...
8342    Recently lot student enrolled Berland State Un...
Name: problem_statement, Length: 6819, dtype: object

In [6]:
topics

0                                        [implementation]
1                                    [binarysearch, math]
2                  [dfsandsimilar, graphs, shortestpaths]
3                                                   [dsu]
4       [combinatorics, dfsandsimilar, dsu, graphs, gr...
                              ...                        
8338                                       [greedy, math]
8339       [bruteforce, geometry, greedy, implementation]
8340    [datastructures, dp, greedy, implementation, s...
8341              [binarysearch, combinatorics, dp, math]
8342                                         [bruteforce]
Name: problem_tags, Length: 6819, dtype: object

## Vetorizando

In [7]:
max_df = 0.8
min_df = 0.05
vec = CountVectorizer(max_df=max_df, min_df=min_df, ngram_range=(1,3))

In [8]:
X = vec.fit_transform(text)
X.shape

(6819, 399)

## Agrupando

In [9]:
# Pegando quantidade de topicos

total_topics = set()
topics = topics.apply(lambda x: [total_topics.add(i) for i in x])

In [10]:
n_total_topics = len(total_topics)
print(f'Número de Tópicos: {n_total_topics}')
total_topics

Número de Tópicos: 37


{'*specialproblem',
 '2-sat',
 'binarysearch',
 'bitmasks',
 'bruteforce',
 'chineseremaindertheorem',
 'combinatorics',
 'constructivealgorithms',
 'datastructures',
 'dfsandsimilar',
 'divideandconquer',
 'dp',
 'dsu',
 'expressionparsing',
 'fft',
 'flows',
 'games',
 'geometry',
 'graphmatchings',
 'graphs',
 'greedy',
 'hashing',
 'implementation',
 'interactive',
 'math',
 'matrices',
 'meet-in-the-middle',
 'numbertheory',
 'probabilities',
 'schedules',
 'shortestpaths',
 'sortings',
 'strings',
 'stringsuffixstructures',
 'ternarysearch',
 'trees',
 'twopointers'}

In [29]:
n_components = n_total_topics
max_iter = 100

id2word = Dictionary(text.str.split())
corpus = [id2word.doc2bow(txt) for txt in text.str.split()]

lda = LdaModel(corpus=corpus, num_topics=n_components, id2word=id2word, iterations=max_iter)

In [12]:
lda_matrix = lda.fit_transform(X)

In [13]:
top_word = lda.components_
terms = vec.get_feature_names_out()

for i, comp in enumerate(top_word):
    top_terms_key = zip(terms, comp)
    top_terms_key= sorted(top_terms_key, key = lambda t: t[1], reverse=True)[:10]
    top_terms_list= list(dict(top_terms_key).keys())
    print("Topic "+str(i)+": ",top_terms_list)

Topic 0:  ['vertex', 'edge', 'graph', 'connected', 'number vertex', 'given', 'two', 'pair', 'guaranteed', 'one']
Topic 1:  ['operation', 'perform', 'following', 'given', 'zero', 'take', 'get', 'allowed', 'two', 'equal']
Topic 2:  ['example', 'maximum', 'second', 'possible', 'first example', 'second example', 'one', 'choose', 'third', 'maximum number']
Topic 3:  ['point', 'coordinate', 'distance', 'answer', 'xi', 'absolute', 'two', '10', 'located', 'correct']
Topic 4:  ['move', 'left', 'right', 'position', 'one', 'place', 'left right', 'second', 'put', 'end']
Topic 5:  ['print', 'one', 'order', 'line print', 'solution', 'multiple', 'answer', 'part', 'possible', 'must']
Topic 6:  ['city', 'pair', 'connected', 'two', 'one', 'possible', 'way', 'different', 'every', 'numbered']
Topic 7:  ['000', 'index', 'task', 'written', '000 number', 'distinct', 'order', 'last', 'value', 'sum']
Topic 8:  ['le', 'cdot', '10', 'cdot 10', 'le le', 'le cdot', 'le cdot 10', 'le le cdot', 'integer le', 'intege

## Validando

In [31]:
new_model = CoherenceModel(model=lda, texts=text, corpus=corpus, dictionary=id2word, coherence="u_mass")
print(new_model.get_coherence())

# todo -> entender metrica de coerencia
#      -> usar dentro dos atributos o que ja foi calculado no CountVectorizer

-2.9964287455614778


## Visualizando

In [15]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

panel = pyLDAvis.sklearn.prepare(lda, X, vec, mds='tsne')
panel
# todo 
# mudar para o modelo do gensim

c:\Users\Bruno\Desktop\UNB\PIBIC\classificador-cf\venv\lib\site-packages\pyLDAvis\_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=               x         y  topics  cluster      Freq
topic                                                
13     18.659094  0.155131       1        1  5.987317
22     18.818102 -0.232593       2        1  5.752243
26     18.663433 -1.198791       3        1  5.396221
5      19.333208 -0.619409       4        1  4.990422
14     19.067701 -0.517152       5        1  4.782522
15     19.274958 -0.990110       6        1  4.673335
35     18.879330 -0.588669       7        1  4.442448
2      19.012224 -0.775582       8        1  4.326284
28     19.532385 -0.108069       9        1  3.689629
25     19.456440 -0.809560      10        1  3.566710
4      19.312487 -1.215634      11        1  3.296662
30     19.038534 -1.016535      12        1  3.150596
10     18.463573  0.089084      13        1  3.078787
12     19.467278 -0.431812      14        1  2.917560
27     18.744215 -0.125753      15        1  2.783947
8      18.650055  0.236841      16        1  2.676855
19     18.334467 -0.610279      17        1  2.668416
29     18.900902 -1.299911      18        1  2.651946
3      19.286470 -0.407373      19        1  2.457787
17     18.382362 -0.596682      20        1  2.329974
0      19.370184  0.029513      21        1  2.325077
24     18.615105 -0.186486      22        1  1.930917
6      19.577703 -0.662487      23        1  1.834311
33     19.433201 -0.589767      24        1  1.817179
34     18.096691 -0.686126      25        1  1.798112
32     19.508173 -0.386498      26        1  1.775816
36     19.912725  0.179628      27        1  1.750267
16     19.666111 -1.086431      28        1  1.740157
18     19.228895  0.533913      29        1  1.413910
9      19.925827  0.131420      30        1  1.410543
11     19.330296 -1.732451      31        1  1.310619
1      19.808697 -0.792322      32        1  1.160534
21     20.052176 -0.382281      33        1  1.053162
20     19.275227 -1.330238      34        1  0.857176
7      20.284105 -1.005544      35        1  0.825554
31     20.091953 -1.470758      36        1  0.773716
23     20.099659 -0.167032      37        1  0.603290, topic_info=         Term          Freq         Total Category  logprob  loglift
190        le  13556.000000  13556.000000  Default  30.0000  30.0000
348    string   6981.000000   6981.000000  Default  29.0000  29.0000
326  sequence   4400.000000   4400.000000  Default  28.0000  28.0000
354      test   6524.000000   6524.000000  Default  27.0000  27.0000
370      time   4862.000000   4862.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
265       one     72.902752  13639.961758  Topic37  -3.9360  -0.1211
119     every     14.706261   1677.506601  Topic37  -5.5368   0.3737
374       two     23.726542   9835.657516  Topic37  -5.0585  -0.9166
353      task     11.882524   2044.432416  Topic37  -5.7500  -0.0373
74   consists     11.420113   1346.161900  Topic37  -5.7897   0.3409

[1470 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0        17  0.044825         000
0        35  0.953728         000
1        35  0.998855  000 number
2         1  0.360453          10
2         3  0.003978          10
...     ...       ...         ...
398      27  0.082535        zero
398      28  0.067796        zero
398      30  0.037583        zero
398      31  0.194546        zero
398      32  0.114959        zero

[2994 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 23, 27, 6, 15, 16, 36, 3, 29, 26, 5, 31, 11, 13, 28, 9, 20, 30, 4, 18, 1, 25, 7, 34, 35, 33, 37, 17, 19, 10, 12, 2, 22, 21, 8, 32, 24])